##### Null Pointer 역참조 (p.120)

In [ ]:
from django.shortcuts import render
from xml.sax import make_parser, handler
import os

def parse_xml(request):
    filename = request.POST.get('filename')
    # filename이 None 인지 체크
    if filename is None or filename.strip() == "":
        raise ValueError("파일 이름이 없습니다.")

    if (filename.count('.') > 0):
        name, ext = os.path.splitext(filename)
    else:
        ext = ''
        
    if ext == '.xml':
        parser = make_parser()
        ......
        sax_handler = Handler()
        parser.setContentHandler(sax_handler)
        parser.parse(filename)
        return sax_handler.root

##### 부적절한 자원 해제 (p.123)

In [ ]:
import sys
def get_config():
    lines = None
    try:
        f = open('config.cfg')
        lines = f.readlines()
    except Exception as e:
        ...
    finally:
        # try 절에서 할당한 자원은
        # finally절에서 시스템에 반환을 해 주어야 한다
        f.close()
        return lines

In [ ]:
# with 절을 빠져나갈 때 f를 시스템에 반환
with open('filename.py') as f:
	print(f.read())

##### 신뢰할 수 없는 데이터의 역직렬화 (p.126)

In [ ]:
import hmac
import hashlib
import pickle
from django.shortcuts import render

def load_user_object(request):
    hash_pickle = bytes(request.POST.get('hash_pickle', ''), encoding = "utf-8")
    userinfo = bytes(request.POST.get('userinfo', ''), encoding = "utf-8")
    # HMAC 검증을 위한 비밀키는 안전하게 저장하여 사용
    m = hmac.new(key=b’secret_key’, digestmod=hashlib.sha512)
    m.update(userinfo)
    # 전달받은 해시값(hash_pickle)과 직렬화 데이터(userinfo)의
    # 해시값을 비교하여 검증
    if hmac.compare_digest(m.digest(), hash_pickle):
        user_obj = pickle.loads(userinfo_pickle)
        return render(request, '/load_user_obj.html', {'obj':user_obj})
    else:
        return render(request, '/error.html', {'error':'직렬화 오류입니다.'})